## Подключение к БД

Загрузим необходимые для работы библиотеки

In [ ]:
import warnings

import pandas as pd
import optuna
import torch
import torch.nn as nn
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px

from sqlalchemy import create_engine 
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from catboost import CatBoostClassifier

warnings.filterwarnings('ignore')

In [ ]:
db_config = {
'user': 'praktikum_student', # имя пользователя,
'pwd': 'Sdf4$2;d-d30pp', # пароль,
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения,
'db': 'data-science-vehicle-db' # название базы данных,
}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)

In [ ]:
engine = create_engine(connection_string) 

## Первичное исследование таблиц

На начальном этапе выгрузим имеющиеся таблицы, а также посмотрим, какие типы данных в них встречаются и имеются ли в данных пропуски. 

In [ ]:
def introduction_to_data(df):
    parameters = dict()
    parameters['column name'] = list(df.columns)
    parameters['type'] = list(df.dtypes)
    parameters['non-null'] = list(df.count())
    parameters['null_quantity'] = list(df.isnull().sum())
    parameters['null_percentage'] = list((df.isnull().sum())/(df.count()+df.isnull().sum())*100)
    
    info = pd.DataFrame(parameters)

    return  display(info)

In [ ]:
query = '''
SELECT *
FROM parties;
'''

df_first_look = pd.read_sql_query(query, con=engine) 

df_first_look

In [ ]:
info = introduction_to_data(df_first_look)

info

In [ ]:
del df_first_look

In [ ]:
query = '''
SELECT *
FROM collisions;
'''

df_first_look = pd.read_sql_query(query, con=engine) 

df_first_look

In [ ]:
info = introduction_to_data(df_first_look)

info

In [ ]:
del df_first_look

In [ ]:
query = '''
SELECT *
FROM vehicles;
'''

df_first_look = pd.read_sql_query(query, con=engine) 

df_first_look

In [ ]:
info = introduction_to_data(df_first_look)

info

In [ ]:
del df_first_look

In [ ]:
query = '''
SELECT *
FROM case_ids;
'''

df_first_look = pd.read_sql_query(query, con=engine) 

df_first_look

In [ ]:
info = introduction_to_data(df_first_look)

info

In [ ]:
del df_first_look

Исходя из предварительного анализа можно сделать следующие выводы:
1. Все таблицы имеют данные. При этом по отдельным полям обнаружено значительное количество пропусков (необходимо принять во внимание при подготовке данных для анализа и обучения моделей). 
2. Количество таблиц соответствует условию задачи. 
3. Таблицы связаны единым ключом - полем `case_id`.

##  Статистический анализ факторов ДТП

### Распределение ДТП по месяцам

In [ ]:
query = '''
SELECT DATE_PART('month', collision_date) as month,
       COUNT(DISTINCT case_id)
FROM collisions
GROUP BY month;
'''

df_accidents = pd.read_sql_query(query, con=engine) 

df_accidents

In [ ]:
fig_1 = go.Figure()

fig_1.add_trace(
    go.Bar(
        x = ['January','February','March','April','May','June','July','August','September','October','November','December'], 
        y = df_accidents['count'],
        marker_color='rgb(0,0,100)'
    )
)

fig_1.update_layout(title="Quantity of accidents per month",
                    xaxis_title="Month",
                    yaxis_title="Quantity, u",
                   )

fig_1.show()

In [ ]:
del df_accidents

Исходя из данных, представленных на графике, можно заключить, что в первые пять месяцев происходит больше аварий, чем в остальные. Попробуем найти причины такого распределения: 
1. Высокая аварийность в первые 3 месяца может быть обусловлена неблагоприятными погодными условиями, ОДНАКО в таком случае в декабре тоже должен наблюдаться рост количества ДТП (если только не списать отсутствие роста на рождественские праздники, в которые люди часто проводят время дома с семьей/друзьями). 
2. Снижение числа ДТП в летние месяцы можно объяснить началом отпусков и общим снижением траффика на дорогах, что отчасти может подтверждаться ростом числа ДТП в октябре. 

### Постановка задач для дополнительного анализа

Для анализа было бы интересно рассмотреть следующие задачи:
1. Распределение ДТП в зависимости от погодных условий на момент ДТП.
2. Распределение ДТП в зависимости от состояния проезжей части на момент ДТП.
3. Категория нарушения, физическое состояние участника и тяжесть последствий ДТП (для этого необходимо связать `collisions` и `parties` - <b>будет рассмотрена отдельно</b>).
4. Тип аварии и тяжесть последствий ДТП.
5. Распределение ДТП в зависимости от времени суток, физическое состояние участника и тяжесть последствий ДТП (для этого необходимо связать `collisions` и `parties` - <b>будет рассмотрена отдельно</b>).
6. Распределение ДТП в зависимости от дорожных условий (в таблице `collision` поле `ROAD_CONDITION`) на момент ДТП.

#### Анализ категории нарушения, физического состояния участника и тяжести последствий ДТП

In [ ]:
query = '''
SELECT COUNT (col.case_id),
       primary_collision_factor AS factor,
       collision_damage AS damage,
       party_drug_physical AS participant_condition
FROM collisions AS col
INNER JOIN parties AS par ON col.case_id = par.case_id
WHERE party_drug_physical <> '-' AND primary_collision_factor <> '-'
GROUP BY factor, damage, participant_condition;
'''

df_accidents = pd.read_sql_query(query, con=engine) 

df_accidents

In [ ]:
fig = px.bar(df_accidents, x="participant_condition", y="count", color="damage", barmode="group", facet_col="factor")

fig.update_layout(title="Distribution of accidents per factor and participant's condition",
                   yaxis_title="Quantity of accidents, u",
                    legend=dict(x=0.5, y=-0.7, xanchor="center", orientation="h"),
                   )
fig.show()

По результатам анализа данного графика можно сделать следующие выводы:
1. Основной причиной ДТП является нарушение правил дорожного движения. 
2. По имеющимся данным невозможно определить влияние состояния участника на вероятность ДТП - для большинства участников состояние либо не оценивалось, либо неизвестно. 

In [ ]:
del df_accidents

#### Анализ времени суток, физического состояния участника и тяжести последствий ДТП

In [ ]:
query = '''
SELECT COUNT (col.case_id),
       DATE_PART('hour', collision_time) AS hour,
       CASE 
            WHEN DATE_PART('hour', collision_time)>=4 AND DATE_PART('hour', collision_time)<10 THEN 'morning'
            WHEN DATE_PART('hour', collision_time)>=10 AND DATE_PART('hour', collision_time)<17 THEN 'day'
            WHEN DATE_PART('hour', collision_time)>=17 AND DATE_PART('hour', collision_time)<23 THEN 'evening'
            ELSE 'night'
       END,
       collision_damage AS damage,
       party_drug_physical AS participant_condition
FROM collisions AS col
INNER JOIN parties AS par ON col.case_id = par.case_id
WHERE party_drug_physical <> '-' AND collision_time IS NOT NULL
GROUP BY hour, damage, participant_condition;
'''

df_accidents = pd.read_sql_query(query, con=engine) 

df_accidents

In [ ]:
fig = px.bar(df_accidents, x="participant_condition", y="count", color="damage", barmode="group", facet_col="case")

fig.update_layout(title="Distribution of accidents per time and participant's condition",
                   yaxis_title="Quantity of accidents, u",
                    legend=dict(x=0.5, y=-0.7, xanchor="center", orientation="h"),
                   )
fig.show()

По результатам анализа данного графика можно сделать следующие выводы:
1. Основной количество ДТП приходится на период с 10:00 до 23:00. 
2. Для утренних и ночных часов можно заметить рост числа уснувших за рулем.
3. Для дневных и вечерних часов можно заметитт рост числа водителей под воздействием лекарств. 
4. В целом по имеющимся данным сложно определить влияние состояния участника на вероятность ДТП - для большинства участников состояние либо не оценивалось, либо неизвестно. 

In [ ]:
del df_accidents

### Предобработка данных

Датасет для моделей будет включать в себя следующие поля:
1. Дата происшествия (`collision_date`) - в будние и выходные дни количество ДТП может разниться, так же как и по месяцам года. 
2. Время происшествия (`collision_time`) - в определенные часы траффик интенсивнее и, как следствие, количество ДТП возрастает. 
3. Является ли место происшествие перекрёстком (`intersection`) - на автомобильных дорогах и внутригородских магистралях/улицах часто встречаются места концентрации ДТП, их полезно выявлять. 
4. Погода (`weather_1`) - погодные условия также влияют на вероятность возникновения ДТП.
5. Основной фактор аварии (`primary_collision_factor`).
6. Состояние дороги (`road_surface`) - состояние проезжей части важно учитывать при анализе ДТП. 
7. Освещение (`lighting`) - освещенность также является важным фактором при учете риска возникновения ДТП. 
8. Номер географических районов (`county_city_location`) - данное поле может косвенно учитывать климатические условия районов, что немаловажно. 
9. Расстояние от главной дороги (`distance`) - на главных дорогах траффик выше и, как следствие, выше риск возникновения ДТП. 
10. Тип дороги (`location_type`) - местоположеное важно учитывать при анализе риска возникновения ДТП. 
11. Категория нарушения (`pcf_violation_category`).
12. Тип аварии (`type_of_collision`). 
13. Дорожное состояние (`road_condition_1`) - состояние дорожного покрытия может влиять на риск возникновения ДТП. 
14. Устройство управления (`control_device`) - техническое состояние ТС необходимо учитывать.
15. Виновность участника (`at_fault`) - целевой параметр. 
16. Состояние участника (`party_drug_physical`) - исходя из постороенного выше графика полезно учесть это поле. 
17. Наличие телефона в автомобиле (`cellphone_in_use`) - отсутствие безпроводной связи отвлекает водителя от управления ТС и повышает риск возникновения ДТП. 
18. Возраст автомобиля (`vehicle_age`) - чем старше ТС, тем выше риск наличия или возникновения неисправностей. 

In [ ]:
query = '''
SELECT col.case_id,
       DATE_PART('month', collision_date) AS collision_date,
       DATE_PART('hour', collision_time) AS collision_time,
       intersection,
       weather_1 AS weather,
       primary_collision_factor,
       road_surface,
       lighting,
       county_city_location,
       distance,
       location_type,
       pcf_violation_category,
       type_of_collision,
       road_condition_1 AS road_condition,
       control_device,
       at_fault,
       party_drug_physical,
       cellphone_in_use,
       vehicle_age
FROM collisions AS col
INNER JOIN parties AS par ON col.case_id = par.case_id
INNER JOIN vehicles AS veh ON col.case_id = veh.case_id
WHERE party_type = 'car'
      AND collision_damage <> 'scratch'
      AND DATE_PART('year', collision_date) = 2012
'''

df_initial = pd.read_sql_query(query, con=engine) 

df_initial

In [ ]:
info = introduction_to_data(df_initial)

info

### Работа с пропусками

Для полей `collision_time`, `intersection`, `weather`, `primary_collision_factor`, `road_surface`, `lighting`, `pcf_violation_category`, `type_of_collision`, `road_condition`, `control_device` суммарная доля пропусков составляет 4,42% - удалим эти пропуски из датасета. 

In [ ]:
df_initial = df_initial.dropna(
    subset=[
        'collision_time',
        'intersection',
        'weather',
        'primary_collision_factor',
        'road_surface',
        'lighting',
        'pcf_violation_category',
        'type_of_collision',
        'road_condition',
        'control_device'
    ]
    )

В остальных полях доля пропусков слишком велика, чтобы их можно было просто удалить.

1 . В полях `location_type` и `party_drug_physical` пропуски поместим в отдельную категорию "Unknown" и "G" соответственно.
2 . В поле `cellphone_in_use` поставим 0 - примем, что автомобиль по умолчанию не оборудован системой беспроводной связи.
3 . В поле `vehicle_age` заменим пропуски на медианное значение.

In [ ]:
df_initial['location_type'] = df_initial['location_type'].fillna('unknown')
df_initial['party_drug_physical'] = df_initial['party_drug_physical'].fillna('G')
df_initial['cellphone_in_use'] = df_initial['cellphone_in_use'].fillna(0)
df_initial['vehicle_age'] = df_initial['vehicle_age'].fillna(df_initial['vehicle_age'].median())

In [ ]:
info = introduction_to_data(df_initial)

info

### Поиск дубликатов

Проверим, содержатся ли в полях с категориальными признаками дубли

In [ ]:
cat_columns = ['weather',
               'primary_collision_factor',
               'road_surface',
               'lighting',
               'location_type',
               'pcf_violation_category',
               'type_of_collision',
               'road_condition',
               'control_device',
               'party_drug_physical'
              ]

for i in cat_columns:
    print(f'Поле {i}')
    print(df_initial[i].unique())
    print()

Неявных дубликатов не обнаружено.

Проверим, не попали ли в таблицу явные дубликаты. 

In [ ]:
print(len(df_initial))

df_initial = df_initial.drop_duplicates()

print(len(df_initial))

<b> Как мы видим, в датасете были явные дубликаты. </b>

После удаления явных дубликатов поле с id каждого случая также удалим из датасета

In [ ]:
df_initial = df_initial.drop('case_id', axis=1)

### Поиск выбросов

In [ ]:
def outliers(df, column):
    initial_df = df
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    upper = Q3+1.5*IQR
    lower = Q1-1.5*IQR
    
    new_df = df.loc[(df[column] < lower) | (df[column] > upper)]
    
    initial = len(initial_df)
    new = len(new_df)
    ratio = (new/initial)*100
    
    maximum = df[column].max()
    minimum = df[column].min()
    
    print(
        "Столбец {}\nМаксимальное значение = {:.0f},\nМинимальное значение = {:.0f},\n"\
        "25-й квантиль = {:.0f},\n75-й квантиль = {:.0f},\n"\
        "Верхняя граница = {:.0f},\nНижняя граница = {:.0f},\nКоличество строк в исходном датасете = {:.0f},\n"\
        "Количество выбросов = {:.0f},\nДоля выбросов = {:.3f}\n".format(column,maximum,minimum,Q1,Q3,upper,lower,initial, new, ratio)
    )

In [ ]:
for i in ['distance','vehicle_age']:
    outliers(df_initial, i)

Как мы видим, в полях `distance` и `vehicle_age` присутствуют выбросы, при этом:
1. В поле `distance` доля выбросов составляет 12% от всего датасета, не будем удалять такой объем датасета. 
2. В поле `vehicle_age` доля выбросов составляет 0.64% - выбросы можно удалить без вреда для остального датасета. 

In [ ]:
df_initial = df_initial.query('vehicle_age <= 13')
df_initial = df_initial.reset_index()

### Статистический анализ данных

Рассмотрим распределение ДТП по каждому из категориальных признаков

In [ ]:
cat_columns = ['weather',
               'primary_collision_factor',
               'road_surface',
               'lighting',
               'location_type',
               'pcf_violation_category',
               'road_condition',
               'type_of_collision'
              ]

for i in cat_columns:
    fig = px.histogram(df_initial, x=i)

    fig.update_layout(title="Quantity of accidents",
                        xaxis_title=i,
                        yaxis_title="Quantity, u",
                       )

    fig.show()

Исходя из анализа данных можно сделать следующие выводы:
1. По графику количества ДТП в зависимости от погоды основная часть ДТП произошла <b>в ясную погоду</b>. 
2. По графику количества ДТП в зависимости от состояния проезжей части основная часть ДТП произошла <b>на сухом покрытии</b>.
3. По графику количества ДТП в зависимости от освещенности основная часть ДТП произошла <b>в светлое время суток</b>.
4. По графику количества ДТП в зависимости от местоположения для значительной части ДТП <b>местоположение неизвестно</b>.
5. По графику количества ДТП в зависимости от состояния дорожной одежды основная часть ДТП произошла <b>на участках без дефектов/препятствий</b>.

<b>При этом видно, что</b>:
1. Большая часть ДТП произошла из-за нарушения ПДД. 
2. Среди нарушений ПДД лидирует категория "Нарушение скоростного режима". 

### Подготовка данных для моделей

In [ ]:
target = df_initial['at_fault']
features = df_initial.drop('at_fault', axis=1)

In [ ]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, 
    target, 
    test_size=0.4, 
    random_state=42,                          
    shuffle=True
)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, 
    target_valid, 
    test_size=0.4, 
    random_state=42,                          
    shuffle=True
)

num = ['distance','vehicle_age']

cat = ['weather',
       'primary_collision_factor',
       'road_surface',
       'lighting',
       'location_type',
       'pcf_violation_category',
       'type_of_collision',
       'road_condition',
       'control_device',
       'party_drug_physical',
       'intersection',
       'control_device'
      ]    


col_transformer = ColumnTransformer(
                    transformers=[
                        ('ss', StandardScaler(), num),
                        ('ohe', OneHotEncoder(drop='first', sparse=False), cat)
                    ],
                    remainder='drop',
                    n_jobs=-1
                    )

In [ ]:
features_train = col_transformer.fit_transform(features_train)
features_valid = col_transformer.transform(features_valid)  
features_test = col_transformer.transform(features_test)  

In [ ]:
features_train_t = torch.FloatTensor(features_train)
features_valid_t = torch.FloatTensor(features_valid)
features_test_t = torch.FloatTensor(features_test)
target_train_t = torch.FloatTensor(target_train.values)
target_valid_t = torch.FloatTensor(target_valid.values)
target_test_t = torch.FloatTensor(target_test.values)

## Создание моделей для оценки водительского риска

Исходя требований заказчика, в ходе выполнения исследования будет решаться задача <b>бинарной классификации</b>.

При этом важно выявлять не только тех пользователей, кто потенциально может попасть в ДТП, но и минимизировать количество <b>ложноотрицательных ответов</b>. Можно было бы принять <b>полноту</b>, однако более информативной была бы метрика <b>ROC-AUC</b>.

Для исследования возьмем следующие модели:
1. Случайный лес.
2. Градиентный бустинг (CatBoost).
3. Нейросеть.

### Случайный лес

In [ ]:
def objective(trial):

    parameters = {
        'max_depth':trial.suggest_int('max_depth', 1, 100),
        'n_estimators':trial.suggest_int('n_estimators', 100, 1000),
        'max_features':trial.suggest_int('max_features', 1, 10),
        'min_samples_leaf':trial.suggest_int('min_samples_leaf', 2, 5),
        'min_samples_split':trial.suggest_int('min_samples_split', 2, 5)
    }

    model = RandomForestClassifier(**parameters)
    model.fit(features_train, target_train)
    predictions = model.predict_proba(features_valid)

    metric = roc_auc_score(target_valid, predictions[:, 1])

    return metric

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best score:', study.best_trial.value)

forest = study.best_trial.value

### Градиентный бустинг

In [ ]:
target_cat = df_initial['at_fault']
features_cat = df_initial.drop('at_fault', axis=1)

In [ ]:
features_train_cat, features_valid_cat, target_train_cat, target_valid_cat = train_test_split(
    features_cat, 
    target_cat, 
    test_size=0.4, 
    random_state=42,                          
    shuffle=True
)

features_valid_cat, features_test_cat, target_valid_cat, target_test_cat = train_test_split(
    features_valid_cat, 
    target_valid_cat, 
    test_size=0.4, 
    random_state=42,                          
    shuffle=True
)

In [ ]:
cat_features = [4,5,6,7,10,11,12,13,14,15]

In [ ]:
def objective(trial):

    parameters = {
        'iterations':trial.suggest_int('iterations', 1, 1000),
        'random_seed':trial.suggest_int('random_seed', 1, 1000),
        'learning_rate':trial.suggest_loguniform('learning_rate',0.1, 0.5),
        'verbose': trial.suggest_categorical('verbose',[False])
    }

    model = CatBoostClassifier(**parameters)
    model.fit(features_train_cat, target_train_cat, cat_features=cat_features)
    predictions = model.predict_proba(features_valid_cat)

    metric = roc_auc_score(target_valid, predictions[:, 1])

    return metric

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best score:', study.best_trial.value)

catboost = study.best_trial.value

### Нейросеть

In [ ]:
n_in_neurons = 67
n_hidden_neurons_1 = 100
n_hidden_neurons_2 = 100
n_hidden_neurons_3 = 100
n_out_neurons = 1

In [ ]:
net = nn.Sequential(
    nn.Linear(n_in_neurons, n_hidden_neurons_1),
    nn.ReLU(),
    nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2),
    nn.ReLU(),
    nn.Linear(n_hidden_neurons_2, n_hidden_neurons_3),
    nn.ReLU(),
    nn.Linear(n_hidden_neurons_3, n_out_neurons),
    nn.Sigmoid()
)

optimizer = torch.optim.SGD(net.parameters(), lr=1.0e-3)

loss = nn.BCELoss()

num_epochs = 1000

for epoch in range(num_epochs):
    net.train()
    optimizer.zero_grad()
    
    preds = net(features_train_t).flatten()
    
    loss_value = loss(preds, target_train_t)
    loss_value.backward()
    optimizer.step()
    
    if epoch % 100 == 0 or epoch == max(range(num_epochs)):
        with torch.no_grad():
            net.eval()
            test_preds = net(features_valid_t).flatten()
            loss_bce = loss(test_preds, target_valid_t)
            test_preds_i = test_preds.int()
            metric = roc_auc_score(target_valid_t.int(), test_preds_i)
            print(loss_bce, metric)

### Batch Normalization

In [ ]:
net_b = nn.Sequential(
    nn.Linear(n_in_neurons, n_hidden_neurons_1),
    nn.ReLU(),
    nn.BatchNorm1d(n_hidden_neurons_1),
    nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2),
    nn.ReLU(),
    nn.BatchNorm1d(n_hidden_neurons_2),
    nn.Linear(n_hidden_neurons_2, n_hidden_neurons_3),
    nn.ReLU(),
    nn.BatchNorm1d(n_hidden_neurons_3),
    nn.Linear(n_hidden_neurons_3, n_out_neurons),
    nn.Sigmoid()
)

optimizer = torch.optim.SGD(net_b.parameters(), lr=1.0e-3)

loss = nn.BCELoss()

num_epochs = 1000

for epoch in range(num_epochs):
    net_b.train()
    optimizer.zero_grad()
    
    preds_b = net_b(features_train_t).flatten()
    
    loss_value = loss(preds_b, target_train_t)
    loss_value.backward()
    optimizer.step()
    
    if epoch % 100 == 0 or epoch == max(range(num_epochs)):
        with torch.no_grad():
            net.eval()
            test_preds_b = net_b(features_valid_t).flatten()
            loss_bce_b = loss(test_preds_b, target_valid_t)
            test_preds_i_b = test_preds_b.int()
            metric_b = roc_auc_score(target_valid_t.int(), test_preds_i_b)
            print(loss_bce_b, metric_b)

### Сравнение результатов предсказаний моделей по ключевой метрике

In [ ]:
fig = ff.create_annotated_heatmap(
        z=[[float("{:.3f}".format(forest)), float("{:.3f}".format(catboost)), float("{:.3f}".format(metric_b))]],
        x=['Random Forest','CatBoost', 'Neural Network'],
        y=['Optuna'],       
        colorscale='ylgnbu',
        colorbar_thickness=30,
        colorbar_ticklen=3
    )
fig.update_layout(title_text='<b>ROC-AUC Value<b>',
                    title_x=0.5,
                    titlefont={'size': 24},
                    width=600, height=450,
                    xaxis_showgrid=False,
                    xaxis={
                        'title': 'Models',
                        'side': 'bottom'
                        },
                    yaxis_showgrid=False,
                    yaxis={
                        'title': 'Hyper parameter tuning',
                        'side': 'bottom'
                        },
                    yaxis_autorange='reversed',                   
                    paper_bgcolor=None,
    )

fig.show()

Как мы видим, наибольшее значение метрики `ROC-AUC` получено при использовании модели "CatBoost". 

### Получение предсказаний на тестовой выборке

In [ ]:
model = CatBoostClassifier(
    iterations=43,
    random_seed=45,
    learning_rate=0.18549429158819739,
    verbose=False,
    custom_loss='AUC'
)

model.fit(
    features_train_cat, target_train_cat,
    cat_features=cat_features,
    eval_set=(features_valid_cat, target_valid_cat),
    verbose=False,
    plot=True
)

In [ ]:
cat_preds = model.predict_proba(features_test_cat)
prob_1 = cat_preds[:, 1]

predictions = []

for i in range(len(features_test_cat)):
    if prob_1[i] >= 0.5:
        pred = 1
    else:
        pred = 0
    predictions.append(pred)

report = classification_report(target_test_cat, predictions, target_names=['Low risk of Accident', 'High risk of Accident'])
print(report)

In [ ]:
conf_m = confusion_matrix(target_test_cat, predictions)

fig = ff.create_annotated_heatmap(
        z=conf_m,
        x=['Low risk of Accident', 'High risk of Accident'],
        y=['Low risk of Accident', 'High risk of Accident'],       
        colorscale='ylgnbu',
        colorbar_thickness=30,
        colorbar_ticklen=3,
    )
fig.update_layout(title_text='<b>Confusion matrix<b>',
                    title_x=0.5,
                    titlefont={'size': 24},
                    width=600, height=550,
                    xaxis_showgrid=False,
                    xaxis={
                        'title': 'Predicted values',
                        'side': 'bottom'
                        },
                    yaxis_showgrid=False,
                    yaxis={
                        'title': 'True values',
                        'side': 'bottom'
                        },
                    yaxis_autorange='reversed',                   
                    paper_bgcolor=None,
    )

fig.show()

<b>По результатам тестирования модели получилось выявлять порядка 63% потенциальных нарушителей.</b>

## Анализ важности факторов ДТП

In [ ]:
importances = pd.DataFrame(model.feature_importances_,
                         features_test_cat.columns)

In [ ]:
fig_feat = px.bar(
    importances,
    x=importances[0],
    )

fig_feat.update_layout(barmode='stack', yaxis={'categoryorder':'total ascending'})
fig_feat.update_xaxes(title_text='Importance')
fig_feat.update_yaxes(title_text='Feature')
fig_feat.update_layout(title_text='Features importance')
fig_feat.update_traces(marker_color='rgb(0, 200, 200)', opacity=0.6)

fig_feat.show()

Как мы видим, 3 наиболее важными факторами являются `type_of_collision`, `pcf_violation_category` и `primary_collision_factor` . Выведем их графики еще раз.

In [ ]:
cat_columns = ['primary_collision_factor',
               'pcf_violation_category',
               'type_of_collision'
              ]

for i in cat_columns:
    fig = px.histogram(df_initial, x=i)

    fig.update_layout(title="Quantity of accidents",
                        xaxis_title=i,
                        yaxis_title="Quantity, u",
                       )

    fig.show()

Исходя из представленных данных можно сделать следующий вывод: <b>основной причиной ДТП является нарушений правил дорожного движения, среди которых наиболее часто встречается нарушение скоростного режима.</b>

Как следствие, на графике с типом столкновений встречается большое число ДТП, в которых один автомобиль въехал в другой сзади или сбоку.

<b>Таким образом, в качестве основной рекомендации следует обратить внимание те участки, где ДТП из-за превышения скоростного режима встречаются чаще (возможно, вести отдельный учет таких участков, либо сразу учитывать, что при движении на скоростных дорогах и магистралях риск таких ДТП в среднем выше). Кроме того, можно вести контроль манеры вождения пользователей и, если они часто превышают скорость, вносить их в категорию водителей, которые потенциально могут устроить ДТП на дороге.</b>